# Introduction au Machine Learning
## Les arbres rencontrent les forêts

In [1]:
import pandas as pd
data = pd.read_csv("data/trees.csv")

## Etape 1 : Importez, nettoyez

In [2]:
### 0.1 Supprimer les lignes sans la target
datac = data[data['ANNEEDEPLANTATION'].isna().map(lambda x : not x)]
print(f"0.1 || removing {data['ANNEEDEPLANTATION'].isna().sum()} rows with NaN target value")
### 1.1 Selections des colonnes
### Colonnes avec TROP de NaN
n = len(data)
datac_NaN_ratio = (datac.isna().sum(axis=0)/n*100).sort_values(ascending=False)
colTooMuchNan = list(datac_NaN_ratio[datac_NaN_ratio > 70].index)
datac = datac.drop(columns=colTooMuchNan)
print(f"1.1 || NaN Ratio : Other Columns have less that 7.009570 % of NaN, removing columns {', '.join(colTooMuchNan)}")
### Colonnes avec 1 seul catégorie/ des catégorie uniques
ndatac = len(datac)
datac_uv = datac.nunique()# Count unique values in each column using nunique()
colOnlyValue = list(datac_uv[datac_uv == 1].index)
datac = datac.drop(columns=colOnlyValue)
### Colonnes avec 1 seul catégorie/ des catégorie uniques
colUnique = list(datac_uv[datac_uv == ndatac].index)
if 'GeoJSON' in colUnique: # We do not remove non categoric (except id)
    colUnique.remove('GeoJSON')    
datac = datac.drop(columns=colUnique)
print(f"1.1 || Removing columns with only 1 value or uniques values : {', '.join(colOnlyValue + colUnique)}")
### Colonnes Description columns 
datac = datac.drop(columns=["SOUS_CATEGORIE_DESC","CODE_PARENT_DESC"])
print(f"1.1 || Removing descritption columns : SOUS_CATEGORIE_DESC, CODE_PARENT_DESC")
### Removes duplicate row (only one)
datac = datac.drop_duplicates()

0.1 || removing 442 rows with NaN target value
1.1 || NaN Ratio : Other Columns have less that 7.009570 % of NaN, removing columns ESSOUCHEMENT, ANNEEABATTAGE, CAUSEABATTAGE, DIAMETREARBRE, REMARQUES, COURRIER, EQUIPE, IDENTIFIANTPLU, INTITULEPROTECTIONPLU, TYPEIMPLANTATIONPLU, RAISONDEPLANTATION, TRAITEMENTCHENILLES, VARIETE
1.1 || Removing columns with only 1 value or uniques values : GENRE, GENRE_DESC, CATEGORIE, CATEGORIE_DESC, ELEM_POINT_ID, CODE, NOM, BIEN_REFERENCE
1.1 || Removing descritption columns : SOUS_CATEGORIE_DESC, CODE_PARENT_DESC


In [3]:
# GPS extraction it is object type since, less precision with float
s = datac["GeoJSON"]
gps_coords = s.str.extractall('([0-9]+\.[0-9]+)').unstack()[0].map(float)
datac['LONGITUDE'] = gps_coords[0]
datac['LATITUDE'] = gps_coords[1]
datac = datac.drop(columns='GeoJSON')

In [4]:
## Exporting to pickle
# datac.to_pickle("data/trees_clean")
import pickle
pickle.dump(datac, open('data/trees_clean.p', 'wb'))